In [ ]:
def consultaGeneral ():
    from py2neo import Graph
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789"))

    import re
    #Cargamos los esquemas de medicina y vivo
    consulta=graph.run('match(i:instancia)-[:rdfs__subClassOf]->(titulo:title)'+
                       'match(i:instancia)-[:rdfs__subClassOf]->(abstract:abstract)'+
                       'match(i:instancia)-[:rdfs__subClassOf]->(link:link)'+
                       'match(i:instancia)-[:rdfs__subClassOf]->(fuente:fuente)'+
                       'match(i:instancia)-[:rdfs__subClassOf]->(autores:autor)'+
                       'match(i:instancia)-[:rdfs__subClassOf]->(keyword:keyword)'+
                       'match(i:instancia)-[:rdfs__subClassOf]->(tituloCategoria:categoriaTitulo)'+
                       'match(i:instancia)-[:rdfs__subClassOf]->(Abstractcategoria:categoriaAbstract)'+ 
                      'return titulo,abstract,link,fuente,autores,autores.nombres,keyword.palabras,tituloCategoria.caracteristicas,Abstractcategoria.caracteristicas').to_data_frame();
    return consulta

In [ ]:
def obtenerautores(caracteristica):
    from py2neo import Graph
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789"))
    
    dict_autor_persona={}
    tam=len(caracteristica)
    for i in range(tam):
        con_autor=graph.run('match(i:autor{nombres:"'+caracteristica[i]+'"})-[:rdfs__subClassOf]->(p:persona) return i,p.nombre').to_data_frame();
        listautor=[]
        for j in con_autor.index:
            listautor.append(con_autor['p.nombre'][j])
        dict_autor_persona[i]=listautor

    import pandas as pd

    df = pd.DataFrame([[key, dict_autor_persona[key]] for key in dict_autor_persona.keys()], columns=['nodo', 'autores'])
    consulta['autores']=df['autores']


In [ ]:
def obtenerkeywords(caracteristica):
    from py2neo import Graph
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789"))
    
    dict_autor_persona={}
    tam=len(caracteristica)
    for i in range(tam):
        con_autor=graph.run('match(i:keyword{palabras:"'+caracteristica[i]+'"})-[:rdfs__subClassOf]->(p:palabra) return i,p.nombre').to_data_frame();
        listautor=[]
        for j in con_autor.index:
            listautor.append(con_autor['p.nombre'][j])
        dict_autor_persona[i]=listautor

    import pandas as pd

    df = pd.DataFrame([[key, dict_autor_persona[key]] for key in dict_autor_persona.keys()], columns=['nodo', 'keywords'])
    consulta['keywords']=df['keywords']



In [ ]:
def obtenerCategoria(caracteristica):
    from py2neo import Graph
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789"))
    
    dict_autor_persona={}
    tam=len(caracteristica)
    for i in range(tam):
        con_autor=graph.run('match(i:categoriaTitulo{caracteristicas:"'+caracteristica[i]+'"})-[:rdfs__subClassOf]->(p:palabraclave) return i,p.nombre').to_data_frame();
        listautor=[]
        for j in con_autor.index:
            listautor.append(con_autor['p.nombre'][j])
        dict_autor_persona[i]=listautor

    import pandas as pd

    df = pd.DataFrame([[key, dict_autor_persona[key]] for key in dict_autor_persona.keys()], columns=['nodo', 'categoriaTitulo'])
    consulta['categoriaTitulo']=df['categoriaTitulo']



In [ ]:
def obtenerCategoriaAbstract(caracteristica):
    from py2neo import Graph
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789"))
    
    dict_autor_persona={}
    tam=len(caracteristica)
    for i in range(tam):
        con_autor=graph.run('match(i:categoriaAbstract{caracteristicas:"'+caracteristica[i]+'"})-[:rdfs__subClassOf]->(p:palabraclave) return i,p.nombre').to_data_frame();
        listautor=[]
        for j in con_autor.index:
            listautor.append(con_autor['p.nombre'][j])
        dict_autor_persona[i]=listautor

    import pandas as pd

    df = pd.DataFrame([[key, dict_autor_persona[key]] for key in dict_autor_persona.keys()], columns=['nodo', 'categoriaAbstract'])
    consulta['categoriaAbstract']=df['categoriaAbstract']


In [ ]:
consulta=consultaGeneral()
obtenerautores(consulta['autores.nombres'])
obtenerkeywords(consulta['keyword.palabras'])
obtenerCategoria(consulta['tituloCategoria.caracteristicas'])
obtenerCategoriaAbstract(consulta['Abstractcategoria.caracteristicas'])

json=consulta.to_json(orient = 'index')
print(json)

# Recomendacion Node Similarity

Se ingresan los sitomas detectados por el terapista.

In [ ]:
def recomNodesimilarity(data):
    from py2neo import Graph
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789")) 
    list_resultados=[]
    for i in range(len(data['caracteristicasT1'])):
        list_resultados.append(graph.run("match(n:categoriaTitulo{caracteristicas:'"+data['caracteristicasT1'][i]+"'})<-[:rdfs__subClassOf]-(i:instancia)-[:rdfs__subClassOf]->(titulo:title)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(abstract:abstract)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(link:link)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(autor:autor)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(fuente:fuente)"+
        "return titulo,abstract,link,autor,fuente").to_data_frame())
    result = pd.concat(list_resultados)  
    result.reset_index(drop=True, inplace=True)
    json=result.to_json(orient = 'index')
    return json
        

    

In [ ]:
def nodesimilarity(listpln):
    from py2neo import Graph
    #Recorrido dentro de la base 
    articulos_obtenidos=[]
    aux_diccionario={}

    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789"))

    for i in range (len(listpln)):
        #print(listpln[i])
        try:
            coincidencia=graph.run('match(n:palabraclave{nombre:"'+listpln[i]+'"})<-[:rdfs__subClassOf]-(u:categoriaTitulo)<-[:rdfs__subClassOf]-(i:instancia)-[:rdfs__subClassOf]->(t:title) return n,u,i,id(t),t').to_data_frame()
            #print("Articulos que contienen:",listpln[i])
            #print(list(coincidencia.get("id(t)")))
            aux_pesos=list(coincidencia.get("id(t)"))
            #Este for permite guardar todos lo id de los articulos relacionados con esa palabra
            for i in range(len(aux_pesos)):
                articulos_obtenidos.append(list(coincidencia.get("id(t)"))[i])
                #print(list(coincidencia.get("t"))[i])
                aux_diccionario[list(coincidencia.get("id(t)"))[i]]=list(coincidencia.get("t"))[i]
            #print("-------------------------")
        except:
            print("No se encontro en la base la palabra ",listpln[i])



    limpieza=list(set(articulos_obtenidos))
    #print(limpieza)
    

    resultados={}
    for i in range(len(limpieza)):
        resultado=graph.run('match(n)<-[x:rdfs__subClassOf]-(i:instancia)-[x1:rdfs__subClassOf]->(k:categoriaTitulo) where ID(n)='+str(limpieza[i])+' return n.title,k.caracteristicas').to_data_frame()
        resultados[resultado['k.caracteristicas'][0]]=resultado['n.title'][0]
    #print(resultados.keys())





    nodesimilarity=graph.run("CALL gds.nodeSimilarity.stream('myGraph20') "+
                             "YIELD node1, node2, similarity "+
                             "RETURN gds.util.asNode(node1).caracteristicas  AS caracteristicasT1, gds.util.asNode(node2).caracteristicas  AS caracteristicasT2, similarity "+
                             "ORDER BY similarity DESCENDING, caracteristicasT1, caracteristicasT2").to_data_frame()


    import pandas as pd

    df = pd.DataFrame()
    list_keys=list(resultados.keys())
    for i in range(len(list_keys)):
        filtrado=nodesimilarity[nodesimilarity['caracteristicasT1'] == list_keys[i]]
        df =pd.concat([df,filtrado])

    df_orden = df.sort_values('similarity',ascending=False)
    df_orden.head()
    #eliminamos lo repetidos
    dataset=df_orden.drop_duplicates(subset=['caracteristicasT1'])  
    #limpiamos los indices
    dataset=dataset.reset_index()
    return dataset
    

In [ ]:
def PLN(texto):
    texto=texto.lower()
    doc = nlp(texto)
    list_importante=[]
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
        if token.pos_=='VERB' or token.pos_=='INTJ' or token.pos_=="ADJ" or token.pos_=="NOUN" or token.pos_=="ADV":
            list_importante.append(token.text)
    #Funcion que permite ordenar el list en forma del abecedario.
    list_importante.sort()
    #print("Palabras a buscar",importante)
    return list_importante

In [ ]:
def buscar(texto):
    list_pln=PLN(texto)
    resultado=nodesimilarity(list_pln)
    salida=recomNodesimilarity(resultado)
    return salida
    

In [ ]:
import pandas as pd
import random
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en')

buscar("""A clinical and histological study of Ullrich's disease (congenital atonic-sclerotic muscular dystrophy""")

# Recomendacion Degree Centrality


In [112]:
def recomDegreeSimilarity(data):
    from py2neo import Graph
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789")) 
    list_resultados=[]
    for i in range(len(data['name'])):
        list_resultados.append(graph.run("match(n:categoriaTitulo{caracteristicas:'"+data['name'][i]+"'})<-[:rdfs__subClassOf]-(i:instancia)-[:rdfs__subClassOf]->(titulo:title)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(abstract:abstract)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(link:link)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(autor:autor)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(fuente:fuente)"+
        "return titulo,abstract,link,autor,fuente").to_data_frame())
    result = pd.concat(list_resultados)  
    result.reset_index(drop=True, inplace=True)
    json=result.to_json(orient = 'index')
    print(json)
    return json
        

In [113]:
def degrecentrality(listpln):
    from py2neo import Graph
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789"))
    
    articulos_obtenidos=[]
    aux_diccionario={}

    for i in range (len(listpln)):
        #print(listpln[i])
        try:
            coincidencia=graph.run('match(n:palabraclave{nombre:"'+listpln[i]+'"})<-[:rdfs__subClassOf]-(u:categoriaTitulo)<-[:rdfs__subClassOf]-(i:instancia)-[:rdfs__subClassOf]->(t:title) return n,u,i,id(t),t').to_data_frame()
            aux_pesos=list(coincidencia.get("id(t)"))
            #Este for permite guardar todos lo id de los articulos relacionados con esa palabra
            for i in range(len(aux_pesos)):
                articulos_obtenidos.append(list(coincidencia.get("id(t)"))[i])
                aux_diccionario[list(coincidencia.get("id(t)"))[i]]=list(coincidencia.get("t"))[i]
        except:
            print("No se encontro en la base la palabra ",listpln[i])
    limpieza=list(set(articulos_obtenidos))
    #print(limpieza)
    resultados={}
    for i in range(len(limpieza)):
        resultado=graph.run('match(n)<-[x:rdfs__subClassOf]-(i:instancia)-[x1:rdfs__subClassOf]->(k:categoriaTitulo) where ID(n)='+str(limpieza[i])+' return n.title,k.caracteristicas').to_data_frame()
        resultados[resultado['k.caracteristicas'][0]]=resultado['n.title'][0]
    #print('Categorias Encotradas')
    #print(resultados.keys())

    degreecentrality=graph.run("CALL gds.degree.stream('myGraph12') "+
                             "YIELD nodeId, score "+
                             "RETURN gds.util.asNode(nodeId).caracteristicas AS name, score AS followers "+
                             "ORDER BY followers DESC, name DESC").to_data_frame()
    
   

    import pandas as pd
    df = pd.DataFrame()
    list_keys=list(resultados.keys())
    for i in range(len(list_keys)):
        filtrado=degreecentrality[degreecentrality['name'] == list_keys[i]]
        df =pd.concat([df,filtrado])

    df_orden = df.sort_values('followers',ascending=False)
    df_orden.head()
    #eliminamos lo repetidos
    #dataset=df_orden.drop_duplicates(subset=['caracteristicasT1'])  
    #limpiamos los indices
    dataset=df_orden.reset_index()
    dataset=dataset.reset_index()
    return dataset

    



In [114]:
def PLN(texto):
    texto=texto.lower()
    doc = nlp(texto)
    list_importante=[]
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
        if token.pos_=='VERB' or token.pos_=='INTJ' or token.pos_=="ADJ" or token.pos_=="NOUN" or token.pos_=="ADV":
            list_importante.append(token.text)
    #Funcion que permite ordenar el list en forma del abecedario.
    list_importante.sort()
    print(list_importante)
    return list_importante

In [115]:
def buscar(texto):
    list_pln=PLN(texto)
    resultado=degrecentrality(list_pln)
    salida=recomDegreeSimilarity(resultado)
    return salida

In [116]:
import pandas as pd
import random
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en')

buscar("""A clinical and histological study of Ullrich's disease (congenital atonic-sclerotic muscular dystrophy""")

['atonic', 'clinical', 'congenital', 'disease', 'dystrophy', 'histological', 'muscular', 'sclerotic', 'study', 'ullrich']
No se encontro en la base la palabra  ullrich
{"0":{"abstract":{"abstract":"0"},"autor":{"nombres":"Autores 452"},"fuente":{"fuente":"ProQuest"},"link":{"link":"http:\/\/dialog.proquest.com\/professional\/docview\/587076612?accountid=173210"},"titulo":{"title":"Seizure disorder in mothers of children with orofacial clefts: a case-control study"}},"1":{"abstract":{"abstract":"0"},"autor":{"nombres":"Autores 408"},"fuente":{"fuente":"ProQuest"},"link":{"link":"http:\/\/dialog.proquest.com\/professional\/docview\/594245560?accountid=173210"},"titulo":{"title":"A comparative study of clinic attendance for pedodontic treatment of children with orofacial clefts"}},"2":{"abstract":{"abstract":"0"},"autor":{"nombres":"Autores 435"},"fuente":{"fuente":"ProQuest"},"link":{"link":"http:\/\/dialog.proquest.com\/professional\/docview\/591149250?accountid=173210"},"titulo":{"titl

'{"0":{"abstract":{"abstract":"0"},"autor":{"nombres":"Autores 452"},"fuente":{"fuente":"ProQuest"},"link":{"link":"http:\\/\\/dialog.proquest.com\\/professional\\/docview\\/587076612?accountid=173210"},"titulo":{"title":"Seizure disorder in mothers of children with orofacial clefts: a case-control study"}},"1":{"abstract":{"abstract":"0"},"autor":{"nombres":"Autores 408"},"fuente":{"fuente":"ProQuest"},"link":{"link":"http:\\/\\/dialog.proquest.com\\/professional\\/docview\\/594245560?accountid=173210"},"titulo":{"title":"A comparative study of clinic attendance for pedodontic treatment of children with orofacial clefts"}},"2":{"abstract":{"abstract":"0"},"autor":{"nombres":"Autores 435"},"fuente":{"fuente":"ProQuest"},"link":{"link":"http:\\/\\/dialog.proquest.com\\/professional\\/docview\\/591149250?accountid=173210"},"titulo":{"title":"Design of a postdoctoral training program in the treatment of children with congenital orofacial clefts"}},"3":{"abstract":{"abstract":"A series of 

# Betweenss Centrality

In [165]:
def recombetweennesCentrality(data):
    from py2neo import Graph
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789")) 
    list_resultados=[]
    for i in range(len(data['caracteristicas'])):
        list_resultados.append(graph.run("match(n:categoriaTitulo{caracteristicas:'"+data['caracteristicas'][i]+"'})<-[:rdfs__subClassOf]-(i:instancia)-[:rdfs__subClassOf]->(titulo:title)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(abstract:abstract)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(link:link)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(autor:autor)"+
        "match(i:instancia)-[:rdfs__subClassOf]->(fuente:fuente)"+
        "return titulo,abstract,link,autor,fuente").to_data_frame())
    result = pd.concat(list_resultados)  
    result.reset_index(drop=True, inplace=True)
    json=result.to_json(orient = 'index')
    #print(json)
    return json

In [166]:
def betweennescentrality(listpln):
    from py2neo import Graph
    graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789"))
    
    articulos_obtenidos=[]
    aux_diccionario={}

    for i in range (len(listpln)):
        #print(listpln[i])
        try:
            coincidencia=graph.run('match(n:palabraclave{nombre:"'+listpln[i]+'"})<-[:rdfs__subClassOf]-(u:categoriaTitulo)<-[:rdfs__subClassOf]-(i:instancia)-[:rdfs__subClassOf]->(t:title) return n,u,i,id(t),t').to_data_frame()
            aux_pesos=list(coincidencia.get("id(t)"))
            #Este for permite guardar todos lo id de los articulos relacionados con esa palabra
            for i in range(len(aux_pesos)):
                articulos_obtenidos.append(list(coincidencia.get("id(t)"))[i])
                aux_diccionario[list(coincidencia.get("id(t)"))[i]]=list(coincidencia.get("t"))[i]
        except:
            print("No se encontro en la base la palabra ",listpln[i])
    
    limpieza=list(set(articulos_obtenidos))
    #print(limpieza)
    resultados={}
    for i in range(len(limpieza)):
        resultado=graph.run('match(n)<-[x:rdfs__subClassOf]-(i:instancia)-[x1:rdfs__subClassOf]->(k:categoriaTitulo) where ID(n)='+str(limpieza[i])+' return n.title,k.caracteristicas').to_data_frame()
        resultados[resultado['k.caracteristicas'][0]]=resultado['n.title'][0]
    #print('Categorias Encotradas')
    #print(resultados.keys())

    betweenesscentrality=graph.run("CALL gds.betweenness.stream('myGraph15') "+
                             "YIELD nodeId, score "+
                             "RETURN gds.util.asNode(nodeId).caracteristicas AS caracteristicas, score "+
                             "ORDER BY caracteristicas ASC").to_data_frame()
    
    import pandas as pd
    df = pd.DataFrame()
    list_keys=list(resultados.keys())
    for i in range(len(list_keys)):
        filtrado=betweenesscentrality[betweenesscentrality['caracteristicas'] == list_keys[i]]
        df =pd.concat([df,filtrado])

    df_orden = df.sort_values('score',ascending=False)
    df_orden.head()
    
    dataset=df_orden.reset_index()
    dataset=dataset.reset_index()
    #print(dataset)

    
    return dataset

In [168]:
def PLN(texto):
    texto=texto.lower()
    doc = nlp(texto)
    list_importante=[]
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
        if token.pos_=='VERB' or token.pos_=='INTJ' or token.pos_=="ADJ" or token.pos_=="NOUN" or token.pos_=="ADV":
            list_importante.append(token.text)
    #Funcion que permite ordenar el list en forma del abecedario.
    list_importante.sort()
    print(list_importante)
    return list_importante

In [169]:
def buscar(texto):
    list_pln=PLN(texto)
    resultado=betweennescentrality(list_pln)
    salida=recombetweennesCentrality(resultado)
    return salida

In [170]:
import pandas as pd
import random
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en')

buscar("""A clinical and histological study of Ullrich's disease (congenital atonic-sclerotic muscular dystrophy""")

['atonic', 'clinical', 'congenital', 'disease', 'dystrophy', 'histological', 'muscular', 'sclerotic', 'study', 'ullrich']
No se encontro en la base la palabra  ullrich


'{"0":{"abstract":{"abstract":"Carbamazepine (CBZ; Tegretol) is an effective anticonvulsant drug. We assessed putative CBZ teratogenicity in a sensitive animal model, the A\\/J mouse. Plasma CBZ levels were determined by gas-liquid chromatography. To achieve the accepted human plasma therapeutic range (TR) of CBZ (8-12 micrograms\\/ml), up to a 6X human oral dose was given. Test Group A received 300 mg pure crystalline CBZ\\/kg, suspended in 1.25% carboxymethylcellulose (CMC), at 6 a.m., noon, 6 p.m., and midnight on gestational day 10 (previously determined to be the peak sensitive time for cleft lip and palate induction in the A\\/J) and at 6 a.m. on day 11. Test Group B received 150 mg\\/kg of an experimental CBZ elixir (Tegretal Sirup) on the same regimen. Plasma levels rose to above TR within 15 min, then fell steadily, exiting the TR by 5 hr postgavage. Test Group C was given the experimental CBZ elixir at 4-h intervals for 36 hr. The rapid rise and fall of CBZ plasma levels agai